In [1]:
import median_filter

In [36]:
import numpy as np
import pandas as pd
from skimage import measure
from scipy import ndimage
from scipy import signal


l = np.arange(480) - 1
l[0] = 0
r = np.arange(480) + 1
r[-1] = r[-2]
b = np.arange(640) - 1
b[0] = 0
u = np.arange(640) + 1
u[-1] = u[-2]


def expand(mask):
    new = mask.copy()
    
    for shift in [l, r]:
        new |= mask[shift, :]
    
    for shift in [b, u]:
        new |= mask[:, shift]

    return new


def find_interesting_pixels(img):
    
    #med = ndimage.median_filter(img, size=40)
    med = median_filter.median_filter(img, 20)
    mask = img > med + 10
    
    labels = measure.label(expand(mask))
    
    return pd.DataFrame(
        [
            [*region.centroid, region.area, region.eccentricity, region.solidity]
            for region in measure.regionprops(labels)
        ],
        columns=['r', 'c', 'area', 'eccentricity', 'solidity']
    )

Example.

In [37]:
from PIL import Image

img = np.asarray(Image.open('data/spotGEO/train/10/1.png')).copy()
find_interesting_pixels(img).shape

(209, 5)

Do it for each image.

In [38]:
import pathlib
from joblib import Parallel, delayed
import tqdm

def f(part, seq, frame):
    img = np.array(Image.open(frame))
    return find_interesting_pixels(img).assign(part=part, sequence=int(seq.name), frame=int(frame.stem))

interesting = Parallel(n_jobs=4)(
    delayed(f)(part, seq, frame)
    for part in ['train', 'test']
    for seq in tqdm.tqdm(list(pathlib.Path(f'data/spotGEO/{part}').glob('*')), position=0)
    for frame in seq.glob('*.png')
)

interesting = pd.concat(interesting)
interesting = interesting.set_index(['part', 'sequence', 'frame']).sort_index()
interesting.to_pickle('data/interesting.pkl')

100%|██████████| 5120/5120 [14:46<00:00,  5.77it/s]


Average number of interesting regions per image.

In [39]:
interesting.groupby(['part', 'sequence', 'frame']).size().mean()

99.26303125

Percentage of pixels this represents.

In [40]:
f'{len(interesting) / (640 * 480 * 31996):%}'

'0.032316%'

Load the provided annotations.

In [41]:
import json
import pandas as pd

sats = []

with open('data/spotGEO/train_anno.json') as f:
    for ann in json.load(f):
        for i, coords in enumerate(ann['object_coords']):
            sats.append({
                'sequence': ann['sequence_id'],
                'frame': ann['frame'],
                'satellite': i + 1,
                'r': int(coords[1] + .5),
                'c': int(coords[0] + .5),
            })
    
sats = pd.DataFrame(sats)
sats = sats.set_index(['sequence', 'frame', 'satellite'])
sats.head()

r    c
sequence frame satellite          
1        1     1          237  502
               2          222  490
               3          129  141
         2     1          214  530
               2          199  518

Now let's annotate each interesting region.

In [42]:
from scipy import optimize
from scipy.spatial import distance

def assign_labels(interesting, satellites):
    
    # Compute the distance between each satellite and each interesting location,
    # thus forming a bipartite graph
    distances = distance.cdist(satellites, interesting)
    
    # Guess which locations correspond to which satellites
    row_ind, col_ind = optimize.linear_sum_assignment(distances)

    # Each satellite is assigned, but some of them may be too distant to be likely
    likely = distances[row_ind, col_ind] < 3
    
    labels = np.full(len(interesting), False, dtype=bool)
    labels[col_ind[likely]] = True
    return labels

Example.

In [43]:
interesting

r           c  area  eccentricity  solidity
part  sequence frame                                                      
test  1        1        7.216981  339.245283   106      0.986314  0.848000
               1       10.751724  264.551724   145      0.980358  0.843023
               1       18.708333   40.666667   144      0.980083  0.862275
               1       20.500000  462.827869   122      0.985141  0.877698
               1       26.525424   65.909605   177      0.993053  0.811927
...                          ...         ...   ...           ...       ...
train 1280     5      466.677419    1.258065    31      0.973495  0.815789
               5      463.000000  290.000000     5      0.000000  1.000000
               5      473.781250   55.875000    32      0.971188  0.780488
               5      476.500000  293.000000     8      0.577350  1.000000
               5      478.285714  107.000000     7      0.377964  1.000000

[3176417 rows x 5 columns]

In [44]:
assign_labels(interesting.loc['train', 1, 1][['r', 'c']], sats.loc[1, 1][['r', 'c']])

array([False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False,  True, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False,  True,
       False, False,  True, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False])

Now assign labels for each frame.

In [45]:
labels = pd.Series(dtype=bool, index=interesting.loc['train'].index)

for (sequence, frame), locations in tqdm.tqdm(interesting.loc['train'].groupby(['sequence', 'frame']), position=0):
    try:
        satellites = sats.loc[sequence, frame]
    except KeyError:
        continue
    labels.loc[sequence, frame] = assign_labels(locations[['r', 'c']], satellites[['r', 'c']])
    
interesting['is_satellite'] = None
interesting.loc['train', 'is_satellite'] = labels.values
interesting.to_pickle('data/interesting.pkl')

100%|██████████| 6400/6400 [00:13<00:00, 462.53it/s]


Determine the amount of satellites that got assigned.

In [46]:
interesting.loc['train']['is_satellite'].sum() / len(sats)

0.6863900044622936

Next, [Solutions.ipynb](Solutions.ipynb).